In [1]:
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')

In [6]:
#encoding:utf-9
from model_v2 import  *
from data import *
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard
#import keras.backend.tensorflow_backend as K
import matplotlib.pyplot as plt

#config = tf.compat.v1.ConfigProto
#config.gpu_options.allow_growth=True
#sess = tf.Session(config=config)
#K.set_session(sess)
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [7]:
if __name__ == '__main__':

    #path to images which are prepared to train a model
    train_path = "data/phyto/train"
    image_folder = "images"
    label_folder = "labels"
    valid_path =  "data/phyto/validation"
    valid_image_folder ="val_imgs"
    valid_label_folder = "val_labels"
    log_filepath = './log'
    flag_multi_class = True
    num_classes = 3
    dp = data_preprocess(train_path=train_path,image_folder=image_folder,label_folder=label_folder,
                         valid_path=valid_path,valid_image_folder=valid_image_folder,valid_label_folder=valid_label_folder,
                         flag_multi_class=flag_multi_class,
                         num_classes=num_classes)

    # train your own model
    train_data = dp.trainGenerator(batch_size=2)
    valid_data = dp.validLoad(batch_size=2)
    test_data = dp.testGenerator()
    model = unet(num_class=num_classes)

    tb_cb = TensorBoard(log_dir=log_filepath)
    model_checkpoint = keras.callbacks.ModelCheckpoint('./model/phyto_model_v1.hdf5', monitor='val_loss',verbose=1,save_best_only=True)
    history = model.fit_generator(train_data,
                                  steps_per_epoch=200,epochs=30,
                                  validation_steps=10,
                                  validation_data=valid_data,
                                  callbacks=[model_checkpoint,tb_cb])

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 512, 512, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 512, 512, 64) 36928       conv2d_24[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 256, 256, 64) 0           conv2d_25[0][0]                  
____________________________________________________________________________________________

Found 198 images belonging to 1 classes.
Found 198 images belonging to 1 classes.
Epoch 1/30
162/200 [=======================>......] - ETA: 12:23 - loss: 0.5354 - accuracy: 0.9664

KeyboardInterrupt: 

In [ ]:
# draw the loss and accuracy curve
    plt.figure(12, figsize=(6, 6), dpi=60)
    plt.subplot(211)
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='val')
    plt.title('loss')
    plt.legend()

    plt.subplot(212)
    plt.plot(history.history['acc'], label='train')
    plt.plot(history.history['val_acc'], label='val')
    plt.title('acc')
    plt.legend()

    plt.show()